# Implementation
### Import modules

In [ ]:
import veloxchem as vlx
import numpy as np

### Set up molecule and basis set

In [ ]:
# molecule and basis

mol_xyz = """12
c2h4-dimer
C         -1.37731        1.01769       -0.71611
C         -0.04211        1.07142       -0.72602
H         -1.96225        1.74636       -0.16458
H         -1.90859        0.23094       -1.24174
H          0.49049        1.84498       -0.18262
H          0.54315        0.32947       -1.25941
C         -1.18880       -1.26220        2.03150
C          0.05470       -0.84410        2.28420
H         -1.37000       -2.18130        1.48410
H          0.91550       -1.41310        1.94850
H         -2.04870       -0.68100        2.34880
H          0.23460        0.08670        2.81250
"""
molecule = vlx.Molecule.from_xyz_string(mol_xyz)
basis = vlx.MolecularBasis.read(molecule, 'def2-svp')

### Set up exciton model

In [ ]:
# exciton model setup

exmod_settings = {
    'fragments': '2',
    'atoms_per_fragment': '6',
    'charges': '0',
    'nstates': '5',
    'ct_nocc': '1',
    'ct_nvir': '1',
}

method_settings = {'dft': 'no'}

exmod_drv = vlx.ExcitonModelDriver()
exmod_drv.update_settings(exmod_settings, method_settings)

### Initializes exciton model Hamiltonian and transition dipoles

In [ ]:
monomer_natoms = list(exmod_drv.natoms)
n_monomers = len(monomer_natoms)
monomer_start_indices = [sum(exmod_drv.natoms[:i]) for i in range(n_monomers)]

npairs = n_monomers * (n_monomers - 1) // 2
total_LE_states = n_monomers * exmod_drv.nstates
total_CT_states = npairs * exmod_drv.ct_nocc * exmod_drv.ct_nvir * 2
total_num_states = total_LE_states + total_CT_states

exmod_drv.H = np.zeros((total_num_states, total_num_states))
exmod_drv.trans_dipoles = np.zeros((total_num_states, 3))
exmod_drv.velo_trans_dipoles = np.zeros((total_num_states, 3))
exmod_drv.magn_trans_dipoles = np.zeros((total_num_states, 3))
exmod_drv.center_of_mass = molecule.center_of_mass()

excitation_ids = exmod_drv.get_excitation_ids()

### Run monomer calculations

In [ ]:
# monomer calculations

monomers_info = [{} for ind in range(n_monomers)]

for ind in range(n_monomers):
    monomer = molecule.get_sub_molecule(monomer_start_indices[ind],
                                        monomer_natoms[ind])
    monomer.set_charge(exmod_drv.charges[ind])
    monomer.check_multiplicity()

    scf_tensors = exmod_drv.monomer_scf(method_settings, ind, monomer, basis)
    tda_results = exmod_drv.monomer_tda(method_settings, ind, monomer, basis,
                                        scf_tensors)

    monomers_info[ind]['mo'] = scf_tensors['C_alpha']
    monomers_info[ind]['exc_energies'] = tda_results['exc_energies']
    monomers_info[ind]['exc_vectors'] = tda_results['exc_vectors']

    one_elec_ints = exmod_drv.get_one_elec_integrals(monomer, basis)
    trans_dipoles = exmod_drv.get_LE_trans_dipoles(monomer, basis,
                                                   one_elec_ints, scf_tensors,
                                                   tda_results)

    # LE states
    for s in range(exmod_drv.nstates):
        h = excitation_ids[ind, ind] + s
        # LE energies
        exmod_drv.H[h, h] = monomers_info[ind]['exc_energies'][s]
        # LE transition dipoles
        exmod_drv.trans_dipoles[h, :] = trans_dipoles['electric'][s]
        exmod_drv.velo_trans_dipoles[h, :] = trans_dipoles['velocity'][s]
        exmod_drv.magn_trans_dipoles[h, :] = trans_dipoles['magnetic'][s]

### Run dimer calculations

In [ ]:
# dimer calculations

for ind_A in range(n_monomers):
    monomer_A = molecule.get_sub_molecule(monomer_start_indices[ind_A],
                                          monomer_natoms[ind_A])
    monomer_A.set_charge(exmod_drv.charges[ind_A])
    monomer_A.check_multiplicity()

    for ind_B in range(ind_A + 1, n_monomers):
        monomer_B = molecule.get_sub_molecule(monomer_start_indices[ind_B],
                                              monomer_natoms[ind_B])
        monomer_B.set_charge(exmod_drv.charges[ind_B])
        monomer_B.check_multiplicity()

        dimer = vlx.Molecule(monomer_A, monomer_B)
        dimer.check_multiplicity()

        mo_A = monomers_info[ind_A]['mo']
        mo_B = monomers_info[ind_B]['mo']

        nocc_A = monomer_A.number_of_alpha_electrons()
        nocc_B = monomer_B.number_of_alpha_electrons()
        nvir_A = mo_A.shape[1] - nocc_A
        nvir_B = mo_B.shape[1] - nocc_B

        nocc = nocc_A + nocc_B
        nvir = nvir_A + nvir_B

        mo = exmod_drv.dimer_mo_coefficients(monomer_A, monomer_B, basis, mo_A,
                                             mo_B)

        dimer_prop = exmod_drv.dimer_properties(dimer, basis, mo)

        dimer_energy = dimer_prop['energy']

        exc_vectors_A = monomers_info[ind_A]['exc_vectors']
        exc_vectors_B = monomers_info[ind_B]['exc_vectors']

        exc_vectors = []

        exc_vectors += exmod_drv.dimer_excitation_vectors_LE_A(
            exc_vectors_A, ind_A, nocc_A, nvir_A, nocc, nvir, excitation_ids)

        exc_vectors += exmod_drv.dimer_excitation_vectors_LE_B(
            exc_vectors_B, ind_B, nocc_A, nvir_A, nocc, nvir, excitation_ids)

        exc_vectors += exmod_drv.dimer_excitation_vectors_CT_AB(
            ind_A, ind_B, nocc_A, nvir_A, nocc, nvir, excitation_ids)

        exc_vectors += exmod_drv.dimer_excitation_vectors_CT_BA(
            ind_A, ind_B, nocc_A, nvir_A, nocc, nvir, excitation_ids)

        sigma_vectors = exmod_drv.dimer_sigma_vectors(dimer, basis, dimer_prop,
                                                      mo, exc_vectors)

        one_elec_ints = exmod_drv.get_one_elec_integrals(dimer, basis)
        trans_dipoles = exmod_drv.get_CT_trans_dipoles(
            dimer, basis, one_elec_ints, mo,
            exc_vectors[exmod_drv.nstates * 2:])

        # CT states
        for i_vec, (c_vec, s_vec) in enumerate(
                zip(exc_vectors[exmod_drv.nstates * 2:],
                    sigma_vectors[exmod_drv.nstates * 2:])):
            # CT energies
            energy = np.vdot(c_vec['vec'], s_vec['vec'])
            exmod_drv.H[c_vec['index'], c_vec['index']] = energy
            # CT transition dipoles
            exmod_drv.trans_dipoles[
                c_vec['index'], :] = trans_dipoles['electric'][i_vec]
            exmod_drv.velo_trans_dipoles[
                c_vec['index'], :] = trans_dipoles['velocity'][i_vec]
            exmod_drv.magn_trans_dipoles[
                c_vec['index'], :] = trans_dipoles['magnetic'][i_vec]

        # LE(A)-LE(B) couplings
        for c_vec in exc_vectors[:exmod_drv.nstates]:
            for s_vec in sigma_vectors[exmod_drv.nstates:exmod_drv.nstates * 2]:
                coupling = np.vdot(c_vec['vec'], s_vec['vec'])
                exmod_drv.H[c_vec['index'], s_vec['index']] = coupling
                exmod_drv.H[s_vec['index'], c_vec['index']] = coupling

        # LE-CT couplings
        for c_vec in exc_vectors[:exmod_drv.nstates * 2]:
            for s_vec in sigma_vectors[exmod_drv.nstates * 2:]:
                coupling = np.vdot(c_vec['vec'], s_vec['vec'])
                exmod_drv.H[c_vec['index'], s_vec['index']] = coupling
                exmod_drv.H[s_vec['index'], c_vec['index']] = coupling

        # CT-CT couplings
        for c_vec in exc_vectors[exmod_drv.nstates * 2:]:
            for s_vec in sigma_vectors[exmod_drv.nstates * 2:]:
                if c_vec['index'] >= s_vec['index']:
                    continue
                coupling = np.vdot(c_vec['vec'], s_vec['vec'])
                exmod_drv.H[c_vec['index'], s_vec['index']] = coupling
                exmod_drv.H[s_vec['index'], c_vec['index']] = coupling

### Get excitation energies and transition dipoles

In [ ]:
# Exciton model energies

eigvals, eigvecs = np.linalg.eigh(exmod_drv.H)

elec_trans_dipoles = np.matmul(eigvecs.T, exmod_drv.trans_dipoles)
velo_trans_dipoles = np.matmul(eigvecs.T, exmod_drv.velo_trans_dipoles)
magn_trans_dipoles = np.matmul(eigvecs.T, exmod_drv.magn_trans_dipoles)

for s in range(total_num_states):
    ene = eigvals[s]
    dip_strength = np.sum(elec_trans_dipoles[s, :]**2)
    f = (2.0 / 3.0) * dip_strength * ene

    velo_trans_dipoles[s, :] /= -ene
    magn_trans_dipoles[s, :] *= -0.5
    R = (-1.0) * np.vdot(velo_trans_dipoles[s, :], magn_trans_dipoles[s, :])

    print(f'S{s+1:<2d}  e={ene:<.8f}  f={f:<.4f}  R={R:<.4f}')